<a href="https://www.kaggle.com/code/adends/titanic-competition-attempt-1?scriptVersionId=164320532" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import matplotlib as plt
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
numerical = ['PassengerId','Pclass','SibSp','Parch','Fare', 'Age']
categorical = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [6]:
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.', expand=True)

print(df[['Name', 'Title']])

                                                  Name Title
0                              Braund, Mr. Owen Harris    Mr
1    Cumings, Mrs. John Bradley (Florence Briggs Th...   Mrs
2                               Heikkinen, Miss. Laina  Miss
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)   Mrs
4                             Allen, Mr. William Henry    Mr
..                                                 ...   ...
886                              Montvila, Rev. Juozas   Rev
887                       Graham, Miss. Margaret Edith  Miss
888           Johnston, Miss. Catherine Helen "Carrie"  Miss
889                              Behr, Mr. Karl Howell    Mr
890                                Dooley, Mr. Patrick    Mr

[891 rows x 2 columns]


From these features we should analyze which are important for analysis. These candidates are Pclass, Sex, Age, SibSp, Parch, Cabin, Embarked.

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [8]:
df.drop(columns=['PassengerId', 'Name', 'Ticket'],axis=1,inplace=True)

In [9]:
encoder = OneHotEncoder()

encoder.fit(df[['Sex','Embarked', 'Cabin', 'Title']])

encoded_data = encoder.transform(df[['Sex', 'Embarked', 'Cabin', 'Title']]).toarray()

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Sex', 'Embarked', 'Cabin', 'Title']))

df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

df = df.drop(['Sex', 'Embarked', 'Cabin', 'Title'], axis=1)

df


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,1,1,19.0,0,0,30.0000,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0,3,NaN,1,2,23.4500,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1,1,26.0,0,0,30.0000,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [10]:
scaler = MinMaxScaler()

scaler.fit(df)

scaled_df = scaler.transform(df)

df=pd.DataFrame(scaled_df, columns=df.columns)

df


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,0.0,1.0,0.271174,0.125,0.000000,0.014151,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.472229,0.125,0.000000,0.139136,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,1.0,0.321438,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.434531,0.125,0.000000,0.103644,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.434531,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,0.5,0.334004,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.233476,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,0.0,1.0,NaN,0.125,0.333333,0.045771,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,1.0,0.0,0.321438,0.000,0.000000,0.058556,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
df.isna().sum()

Survived       0
Pclass         0
Age          177
SibSp          0
Parch          0
            ... 
Title_Mr       0
Title_Mrs      0
Title_Ms       0
Title_Rev      0
Title_Sir      0
Length: 177, dtype: int64

In [12]:
imputer = IterativeImputer()

imputer.fit(df)

imputed_df = imputer.transform(df)

df = pd.DataFrame(imputed_df, columns=df.columns)


In [13]:
df.isna().sum()

Survived     0
Pclass       0
Age          0
SibSp        0
Parch        0
            ..
Title_Mr     0
Title_Mrs    0
Title_Ms     0
Title_Rev    0
Title_Sir    0
Length: 177, dtype: int64

In [14]:
feature_df = df.drop('Survived', axis=1)

pca = PCA(n_components=0.95)

df_pca = pca.fit_transform(feature_df)

df_pca = pd.DataFrame(df_pca, columns=[f'PC{i+1}' for i in range(df_pca.shape[1])])

#df_pca


In [15]:
feature_df

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,1.0,0.271174,0.125,0.000000,0.014151,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.472229,0.125,0.000000,0.139136,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.321438,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.434531,0.125,0.000000,0.103644,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.434531,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.5,0.334004,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
887,0.0,0.233476,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
888,1.0,0.216934,0.125,0.333333,0.045771,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
889,0.0,0.321438,0.000,0.000000,0.058556,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
X, Y = df_pca, df['Survived']

len(X), len(Y)

(891, 891)

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

log_reg = LogisticRegression()

log_reg.fit(X_train,Y_train)

Y_pred = log_reg.predict(X_test)

Y_proba = log_reg.predict_proba(X_test)[:, 1]


In [18]:
# Accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

# Precision
precision = precision_score(Y_test, Y_pred)
print(f"Precision: {precision}")

# Recall
recall = recall_score(Y_test, Y_pred)
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score: {f1}")

# ROC-AUC Score
roc_auc = roc_auc_score(Y_test, Y_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.8246268656716418
Precision: 0.7962962962962963
Recall: 0.7747747747747747
F1 Score: 0.7853881278538812
ROC-AUC Score: 0.8863831984851093


In [19]:
tuning_params = {
    'pca__n_components': [5, 10, 15, 20, 25],  
    'logreg__C': np.logspace(-4, 4, 10),  
    'logreg__solver': ['liblinear', 'saga'],  
    'logreg__penalty': ['l1', 'l2'],  
    'logreg__max_iter': [5000, 10000, 20000]  
}

In [20]:
X = feature_df
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


pipeline = Pipeline([
    ('pca', PCA()),
    ('logreg', LogisticRegression())
])


clf = GridSearchCV(pipeline, tuning_params, cv=5, verbose=0)

best_model = clf.fit(X_train, Y_train)

Y_pred = best_model.predict(X_test)

Y_proba = best_model.predict_proba(X_test)[:, 1]

# Accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

# Precision
precision = precision_score(Y_test, Y_pred)
print(f"Precision: {precision}")

# Recall
recall = recall_score(Y_test, Y_pred)
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score: {f1}")

# ROC-AUC Score
roc_auc = roc_auc_score(Y_test, Y_proba)
print(f"ROC-AUC Score: {roc_auc}")


Accuracy: 0.835820895522388
Precision: 0.8073394495412844
Recall: 0.7927927927927928
F1 Score: 0.8
ROC-AUC Score: 0.8902278074252598
